# keyword_evaluating

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from util import dbhelper

In [2]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1. '여기어때' 사이트 리뷰 데이터의 긍정/부정 예측하기

In [39]:
def sentiment_predict(data):
    # 1차 전처리
#     data = only_ks(data)
#     data = remove_null(data)
#     data = no_repeat(data)
#     data = remove_whitespace(data)
#     data = remove_dupli(data)
#     data = spell_check(data)
#     data = remove_null(data)

    mecab = Mecab()
#     tokenizer = Tokenizer()
    max_len = 425
    loaded_model = load_model('model.h5')
    
    # 2차 전처리
    temp_list = []
    for i in tqdm(range(len(data))):
        temp_dict = {}
        text = mecab.morphs(data['review'][i])
        tokenizer = Tokenizer(14473, oov_token = 'OOV') 
        tokenizer.fit_on_texts(text)
        encoded = tokenizer.texts_to_sequences([text])
        pad_new = pad_sequences(encoded, maxlen = max_len)

        score = float(loaded_model.predict(pad_new))
        temp_dict['tokenized_review'] = text
        if(score > 0.5):
            temp_dict['percent'] = round(score * 100, 2)
            temp_dict['sentiment'] = '긍정'
            
        else:
            temp_dict['percent'] = round((1 - score) * 100, 2)
            temp_dict['sentiment'] = '부정'
        temp_list.append(dict(temp_dict))
    
    temp_df = pd.DataFrame(temp_list)
    new_df = pd.concat([data, temp_df], axis=1)
        
    return new_df

In [42]:
new_df = sentiment_predict(data[:50000])
new_df

100%|█████████████████████████████████████| 50000/50000 [57:00<00:00, 14.62it/s]


,name,score,review,tokenized_review,percent,sentiment
0,뜨랑블루 호텔,10.0,아주 좋은 숙박이에요,"[아주, 좋, 은, 숙박, 이, 에요]",100.00,긍정
1,뜨랑블루 호텔,8.8,바닥에 얼룩이 깨끗하지 않아 아이랑 가기엔 그다지,"[바닥, 에, 얼룩, 이, 깨끗, 하, 지, 않, 아, 아이, 랑, 가, 기, 엔,...",100.00,긍정
2,뜨랑블루 호텔,8.4,두 번째 이용합니다 저도 성호텔에서 근무하는 호텔리어입니다 토요일 일요일 박 이용했...,"[두, 번, 째, 이용, 합니다, 저, 도, 성, 호텔, 에서, 근무, 하, 는, ...",100.00,긍정
3,뜨랑블루 호텔,10.0,자주 오는 곳이라 너무 좋아요 시설도 직원분들 친절도 너무 좋습니다,"[자주, 오, 는, 곳, 이, 라, 너무, 좋, 아요, 시설, 도, 직원, 분, 들...",100.00,긍정
4,뜨랑블루 호텔,10.0,만족합니다 다시 방문할 예정입니다 깨끗하고 친절합니다 감사합니다,"[만족, 합니다, 다시, 방문, 할, 예정, 입니다, 깨끗, 하, 고, 친절, 합니...",100.00,긍정
...,...,...,...,...,...,...
49995,호텔 피제이 명동,10.0,을지로 주변 주차장을 있는 곳을 찾던 중 주차도 가능하며 가격 대비 주변 먹을거리까...,"[을지로, 주변, 주차장, 을, 있, 는, 곳, 을, 찾, 던, 중, 주차, 도, ...",99.98,긍정
49996,호텔 피제이 명동,10.0,룸 컨디션 너무 좋고 룸 업그레이드도 너무 감사합니다 잘 묵고 갈게요,"[룸, 컨디션, 너무, 좋, 고, 룸, 업그레이드, 도, 너무, 감사, 합니다, 잘...",100.00,긍정
49997,호텔 피제이 명동,10.0,침대 푹신하고 무엇보다 친절하세요,"[침대, 푹신, 하, 고, 무엇, 보다, 친절, 하, 세요]",100.00,긍정
49998,호텔 피제이 명동,10.0,자주 이용하는데 침구가 너무 좋아요,"[자주, 이용, 하, 는, 데, 침구, 가, 너무, 좋, 아요]",100.00,긍정


## 2. 정확도 평가

In [64]:
new_df['key_sentiment'] = np.select([new_df.key_score > 3], ['긍정'], default='부정')
new_df['correct'] = np.select([new_df.key_sentiment == new_df.sentiment], ['O'], default='X')
new_df

,name,score,review,tokenized_review,percent,sentiment,key_score,key_sentiment,correct
0,뜨랑블루 호텔,10.0,아주 좋은 숙박이에요,"['아주', '좋', '은', '숙박', '이', '에요']",100.00,긍정,5,긍정,O
1,뜨랑블루 호텔,8.8,바닥에 얼룩이 깨끗하지 않아 아이랑 가기엔 그다지,"['바닥', '에', '얼룩', '이', '깨끗', '하', '지', '않', '아...",100.00,긍정,5,긍정,O
2,뜨랑블루 호텔,8.4,두 번째 이용합니다 저도 성호텔에서 근무하는 호텔리어입니다 토요일 일요일 박 이용했...,"['두', '번', '째', '이용', '합니다', '저', '도', '성', '호...",100.00,긍정,5,긍정,O
3,뜨랑블루 호텔,10.0,자주 오는 곳이라 너무 좋아요 시설도 직원분들 친절도 너무 좋습니다,"['자주', '오', '는', '곳', '이', '라', '너무', '좋', '아요...",100.00,긍정,5,긍정,O
4,뜨랑블루 호텔,10.0,만족합니다 다시 방문할 예정입니다 깨끗하고 친절합니다 감사합니다,"['만족', '합니다', '다시', '방문', '할', '예정', '입니다', '깨...",100.00,긍정,5,긍정,O
...,...,...,...,...,...,...,...,...,...
49995,호텔 피제이 명동,10.0,을지로 주변 주차장을 있는 곳을 찾던 중 주차도 가능하며 가격 대비 주변 먹을거리까...,"['을지로', '주변', '주차장', '을', '있', '는', '곳', '을', ...",99.98,긍정,5,긍정,O
49996,호텔 피제이 명동,10.0,룸 컨디션 너무 좋고 룸 업그레이드도 너무 감사합니다 잘 묵고 갈게요,"['룸', '컨디션', '너무', '좋', '고', '룸', '업그레이드', '도'...",100.00,긍정,5,긍정,O
49997,호텔 피제이 명동,10.0,침대 푹신하고 무엇보다 친절하세요,"['침대', '푹신', '하', '고', '무엇', '보다', '친절', '하', ...",100.00,긍정,5,긍정,O
49998,호텔 피제이 명동,10.0,자주 이용하는데 침구가 너무 좋아요,"['자주', '이용', '하', '는', '데', '침구', '가', '너무', '...",100.00,긍정,5,긍정,O


In [65]:
O_cnt = 0
X_cnt = 0
for i in new_df['correct']:
    if i == 'O':
        O_cnt += 1
    else:
        X_cnt += 1

correct = round((O_cnt / len(new_df)) * 100, 2)
print('O 카운트 :', O_cnt)
print('X 카운트 :', X_cnt)
print('정확도 : {}%'.format(correct))

O 카운트 : 49034
X 카운트 : 966
정확도 : 98.07%
